In [4]:
from mpi4py import MPI

In [ ]:
#EX1: send messages from other processors to 0th.
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank()
p=comm.Get_size()
if my_rank!=0:
    message="Hello from "+str(my_rank)
    comm.send(message,dest=0)
else:
    for Id in range(1,p):
        message=comm.recv(source=Id)
        print("Process 0 receives message from process ",Id,":",message)

In [6]:
#EX2: Compute trapezoid area numerically using parallel computation
comm=MPI.COMM_WORLD
my_rank=comm.Get_rank()
p=comm.Get_size()
def f(x):
    return x*x*x
def Trap(a,b,n,h):
    integral=(f(a)+f(b))/2.0
    x=a
    for i in range(1,int(n)):
        x+=h
        integral+=f(x)
    return integral*h
a=0.0
b=1.0
n=1024
dest=0
total=-1
h=(b-a)/n
local_n=n/p
local_a=a+my_rank*h*local_n
local_b=local_a+local_n*h
integral=Trap(local_a,local_b,local_n,h)
if my_rank==0:
    total=integral
    for Id in range(1,p):
        integral=comm.recv(source=Id)
        print("Process ",my_rank," receives partial result from ",Id,'\n')
        total+=integral
else:
    print("Process ",my_rank," sends partial result \n")
    comm.send(integral,dest=0)
if my_rank==0:
    print("With n=",n," trapezoid integral \n")
    print("from ",a," to ",b," is ",total)

With n= 1024  trapezoid integral 

from  0.0  to  1.0  is  0.2500002384185791
